In [73]:
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np

%matplotlib inline

In [77]:
prev_IG = cv2.imread("highway/input/in000001.jpg")
prev_IG = cv2.cvtColor(prev_IG , cv2.COLOR_BGR2GRAY)
theshold = 18
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
kernel_big = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))

T_P = 0
T_N = 0
F_P = 0
F_N = 0

for i in range(300, 1100):
    I = cv2.imread("highway/input/in%06d.jpg" % i)
    I_GROUND = cv2.imread("highway/groundtruth/gt%06d.png" % i)
    I_GROUND = cv2.cvtColor(I_GROUND, cv2.COLOR_BGR2GRAY)

    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    I_diff = cv2.absdiff(IG, prev_IG)
    I_diff = cv2.medianBlur(I_diff, 5)

    I_diff = (255*( I_diff > theshold)).astype(np.uint8)

    I_open = cv2.morphologyEx(I_diff, cv2.MORPH_OPEN, kernel)
    I_close = cv2.morphologyEx(I_open, cv2.MORPH_CLOSE, kernel_big)
    I_close = cv2.morphologyEx(I_close, cv2.MORPH_CLOSE, kernel_big)
    I_close = cv2.morphologyEx(I_close, cv2.MORPH_CLOSE, kernel_big)

    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(I_close)

    # cv2.imshow("Stream", cv2.cvtColor(I_close, cv2.COLOR_GRAY2BGR))
    cv2.waitKey(10)

    # Display the labels image with appropriate scaling
    # cv2.imshow("Labels", np.uint8(labels / retval * 255))

    I_VIS = np.copy(I)

    # Check if there are any detected objects
    if stats.shape[0] > 1:
        tab = stats[1:, 4]  # Extract the area column excluding the first element
        pi = np.argmax(tab)  # Find the index of the largest object
        pi = pi + 1  # Adjust index to match 'stats'
        
        # Draw bounding box around the largest object
        cv2.rectangle(I_VIS, (stats[pi, 0], stats[pi, 1]), 
                    (stats[pi, 0] + stats[pi, 2], stats[pi, 1] + stats[pi, 3]), 
                    (255, 0, 0), 2)
        
        # Display area of the largest object
        cv2.putText(I_VIS, "%.2f" % stats[pi, 4], 
                    (stats[pi, 0], stats[pi, 1]), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))
        
        # Display index of the largest object at its centroid
        cv2.putText(I_VIS, "%d" % pi, 
                    (int(centroids[pi, 0]), int(centroids[pi, 1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
        
        true_positive_mask = np.logical_and((I_close == 255), (I_GROUND == 255))
        true_positive_sum = np.sum(true_positive_mask)
        T_P = T_P + true_positive_sum

        true_negative_mask = np.logical_and((I_close == 0), (I_GROUND == 0))
        true_negative_sum = np.sum(true_negative_mask)
        T_N = T_N + true_negative_sum

        false_positive_mask = np.logical_and((I_close == 255), (I_GROUND == 0))
        false_positive_sum = np.sum(false_positive_mask)
        F_P = F_P + false_positive_sum

        false_negative_mask = np.logical_and((I_close == 0), (I_GROUND == 255))
        false_negative_sum = np.sum(false_negative_mask)
        F_N = F_N + false_negative_sum

    cv2.imshow("Labels", I_VIS)

    prev_IG = IG

cv2.destroyAllWindows()  # close all windows


precision = T_P / (T_P + F_P) if (T_P + F_P) > 0 else 0
recall = T_P / (T_P + F_N) if (T_P + F_N) > 0 else 0
f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0


print("Presision: ", precision)
print("recall: ", recall)
print("F1: ", f1_score)


Presision:  0.9576557341119745
recall:  0.5362143627101105
F1:  0.6874878347922277
